### Overall structure of the files

In [2]:
import pandas as pd
import numpy as np

customers = pd.read_csv("df_Customers.csv")
orders = pd.read_csv("df_Orders.csv")
order_items = pd.read_csv("df_OrderItems_new.csv")
products = pd.read_csv("df_Products_new.csv")
payments = pd.read_csv("df_Payments_new.csv")


orders.rename(columns={
    'order_purchase_timestamp': 'order_date',
    'order_delivered_timestamp': 'order_delivered_at',
    'order_estimated_delivery_date': 'order_estimated_delivery',
}, inplace=True)

payments.rename(columns={
    'payment_installments': 'installments'
}, inplace=True)

products.rename(columns={
    'product_category_name': 'category'
}, inplace=True)

date_cols = ['order_date', 'order_approved_at', 'order_delivered_at', 'order_estimated_delivery']
for col in date_cols:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors='coerce')

print(" Column names aligned with capstone expectations and date formats converted.")


def inspect_data(df, name):
    print(f"\n--- {name} ---")
    print(df.info())
    print(df.head())
    print("Missing values:\n", df.isnull().sum())

inspect_data(customers, "Customers")
inspect_data(orders, "Orders")
inspect_data(order_items, "Order Items")
inspect_data(products, "Products")
inspect_data(payments, "Payments")


date_cols = ['order_date', 'order_approved_at', 'order_delivered_at', 'order_estimated_delivery']
for col in date_cols:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col], errors='coerce')

order_items['price'] = pd.to_numeric(order_items['price'], errors='coerce')
order_items['freight_value'] = pd.to_numeric(order_items['freight_value'], errors='coerce')

payments['payment_value'] = pd.to_numeric(payments['payment_value'], errors='coerce')
payments['installments'] = pd.to_numeric(payments['installments'], downcast='integer', errors='coerce')

order_items['quantity'] = pd.to_numeric(order_items['quantity'], downcast='integer', errors='coerce')
products['stock_quantity'] = pd.to_numeric(products['stock_quantity'], downcast='integer', errors='coerce')


 Column names aligned with capstone expectations and date formats converted.

--- Customers ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89316 entries, 0 to 89315
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               89316 non-null  object
 1   customer_zip_code_prefix  89316 non-null  int64 
 2   customer_city             89316 non-null  object
 3   customer_state            89316 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.7+ MB
None
    customer_id  customer_zip_code_prefix       customer_city customer_state
0  hCT0x9JiGXBQ                     58125     varzea paulista             SP
1  PxA7fv9spyhx                      3112  armacao dos buzios             RJ
2  g3nXeJkGI0Qw                      4119             jandira             SP
3  EOEsCQ6QlpIg                     18212          uberlandia             MG
4  mVz5LO2Vd6cL                 

### df_Payments

In [3]:
import pandas as pd

orders = pd.read_csv("df_Orders.csv")
payments = pd.read_csv("df_Payments.csv")

orders.rename(columns={
    'order_purchase_timestamp': 'order_date',
    'order_delivered_timestamp': 'order_delivered_at',
    'order_estimated_delivery_date': 'order_estimated_delivery',
}, inplace=True)
payments.rename(columns={
    'payment_installments': 'installments'
}, inplace=True)

payments_with_status = payments.merge(orders[['order_id', 'order_status']], on='order_id', how='left')

unpaid_statuses = ['processing', 'cancelled', 'unavailable']
payments_with_status['payment_status'] = payments_with_status['order_status'].apply(
    lambda status: 'unpaid' if status in unpaid_statuses else 'paid'
)

df_Payments_new = payments_with_status.drop(columns=['order_status'])

df_Payments_new.to_csv("df_Payments_new.csv", index=False)

print("New file saved as df_Payments_new.csv with inferred payment_status column.")


New file saved as df_Payments_new.csv with inferred payment_status column.


### df_Order_Items

In [4]:
import pandas as pd

df_orderitems = pd.read_csv("df_OrderItems.csv")

df_orderitems['quantity'] = 1
unique_orders = df_orderitems['order_id'].nunique()
print(unique_orders)

df_orderitems.to_csv("df_OrderItems_new.csv", index=False)

print(" New file saved as df_OrderItems_new.csv with a 'quantity' column added.")


89316
 New file saved as df_OrderItems_new.csv with a 'quantity' column added.


###  df_Products

In [5]:
import pandas as pd

products = pd.read_csv("df_Products.csv")

products.rename(columns={'product_category_name': 'full_category'}, inplace=True)

products['full_category'] = products['full_category'].fillna('unknown')

products['category'] = products['full_category'].apply(lambda x: x.split('_')[0])
products['sub_category'] = products['full_category'].apply(
    lambda x: ' '.join(x.split('_')[1:]) if '_' in x else 'misc'
)

# Step 4: Optional - Drop full_category column
products.drop(columns=['full_category'], inplace=True)

# Step 5: Save to CSV
products.to_csv("df_Products_new.csv", index=False)

print("New file saved as df_Products_new.csv with extracted 'category' and 'sub_category' columns.")


New file saved as df_Products_new.csv with extracted 'category' and 'sub_category' columns.


### Adding the stock_quantity column

In [6]:
import pandas as pd

# Load your enhanced order items file
df_orderitems = pd.read_csv("df_OrderItems_new.csv")

# Sum quantity ordered per product_id
product_sales = df_orderitems.groupby('product_id')['quantity'].sum().reset_index()
product_sales.rename(columns={'quantity': 'stock_quantity'}, inplace=True)

# Load the products file
df_products = pd.read_csv("df_Products_new.csv")

# Merge estimated stock quantity into products
df_products = df_products.merge(product_sales, on='product_id', how='left')

# Fill missing values (products never ordered) with zero stock
df_products['stock_quantity'] = df_products['stock_quantity'].fillna(0).astype('int')

# Save the result
df_products.to_csv("df_Products_stock_estimated.csv", index=False)

print("Estimated stock_quantity added and saved as df_Products_stock_estimated.csv")


Estimated stock_quantity added and saved as df_Products_stock_estimated.csv


In [7]:
import pandas as pd

# Load the file
df_products = pd.read_csv("df_Products_stock_estimated.csv")

# Drop the specified columns
columns_to_drop = [
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
]
df_products.drop(columns=columns_to_drop, inplace=True)

# Save the cleaned dataset
df_products.to_csv("df_Products_new.csv", index=False)

print("Removed dimension-related columns and saved as df_Products_new.csv")


Removed dimension-related columns and saved as df_Products_new.csv
